In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

# 섹션 이름과 mid 코드 매핑
sections = {
    '최신뉴스': 'n0100',
    '정치': 'n0201',
    '경제': 'n0300',
    '사회': 'n0400',
    '세계': 'n0502',
    'IT/과학': 'n0600'
}

base_url = "https://news.nate.com/recent?mid="

def scrape_nate_news(section_name, section_code):
    url = base_url + section_code
    print(f"==== {section_name} ====")
    print(f"URL: {url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 뉴스 기사 리스트 추출
    articles = soup.select('.mlt01 a')  # 'a' 태그로 연결된 기사들
    for article in articles[:5]:  # 상위 5개 기사만 예시로
        link = article.get('href')
        title = article.get_text(strip=True)
        # 기사 상세페이지에서 이미지 추출
        news_url = urljoin(url, link)
        news_resp = requests.get(news_url)
        news_soup = BeautifulSoup(news_resp.text, 'html.parser')
        
        img_tag = news_soup.find('img', class_='img')  # 이미지 태그 찾기
        img_url = None
        if img_tag and img_tag.get('src'):
            img_src = img_tag['src']
            img_url = urljoin(news_url, img_src)
        
        print("제목:", title)
        print("링크:", news_url)
        if img_url:
            print("이미지:", img_url)
            try:
                display(Image(img_url))
            except Exception as e:
                print("이미지 표시 오류:", e)
        else:
            print("이미지 없음")
        print('-' * 50)

# 모든 섹션 반복
for name, code in sections.items():
    scrape_nate_news(name, code)

==== 최신뉴스 ====
URL: https://news.nate.com/recent?mid=n0100
제목: 사흘 연속 불펜진 방화로 무너진 KIA, 이대로는 한국시리즈 2연패는커녕 한국시리즈 진출조차 '언감생심'이다충격의 스윕패다. 한국시리즈 2연패를 노리는 ‘디펜딩 챔피언’ KIA가 사흘 연속 불펜진의 방화에 주저앉았다. 이대로 가다간 한국시리즈 2연패는커녕 한국시리즈 진출 자...
링크: https://news.nate.com/view/20250725n00368?mid=n0100
이미지 없음
--------------------------------------------------
제목: '심쿵! 이 정도면 연애중?' 보영♥진영…'미지의 서울' 여전한 후유증!사진|엘르 [스포츠서울 | 배우근 기자] 배우 박보영과 박진영의 커플 화보가 대중의 과몰입을 또 불러왔다. 드라마 ‘미지의 서울’ 종영 이후에도 두 사람은 마치 현실 연...
링크: https://news.nate.com/view/20250725n00367?mid=n0100
이미지 없음
--------------------------------------------------
제목: [세상만사] 15만원의 쓰임누군가 지갑에 현금 15만원을 꽂아 준다면 마다할 사람은 거의 없을 것이다. 3만원이 넘는 수박 한 통을 큰맘 먹고 사거나, 비싸서 엄두를 못 냈던 1만5000원짜리 평양냉면...
링크: https://news.nate.com/view/20250725n00366?mid=n0100
이미지 없음
--------------------------------------------------
제목: 제주 해상서 어선 침수…"승선원 8명 모두 구조"어제(24일) 오후 10시 36분께 제주 서귀포시 송악산 인근 해상에서 한림선적 어선 A호(20t·승선원 8명)가 좌초돼 기관실이 침수 중이라며 해경에 구조를 요청했습니다. 서...
링크: https://news.nate.com/view/20250725n003

In [6]:
def download_one_episode(title, no, url):
    import requests
    from bs4 import BeautifulSoup
    import os
    from urllib.parse import urljoin

    # 헤더: User-Agent와 referer 모두 필수!
    req_header = {
        'referer': url,
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    res = requests.get(url, headers=req_header)
    if not res.ok:
        print(f"요청 실패: {url}")
        return
    
    soup = BeautifulSoup(res.text, 'html.parser')
    # 이미지 태그 찾기 (최신 네이버 웹툰 구조 우선, fallback도 포함)
    img_tags = soup.select("img[src*='IMAG01']")
    if not img_tags:
        img_tags = soup.select(".wt_viewer img")  # 예전 구조
    
    if not img_tags:
        print("이미지를 찾지 못했습니다.")
        return
    
    # 저장할 디렉토리: img\제목\회차번호
    dir_name = os.path.join('img', title, str(no))
    os.makedirs(dir_name, exist_ok=True)
    
    for idx, img_tag in enumerate(img_tags, 1):
        img_url = urljoin(url, img_tag['src'])
        try:
            img_res = requests.get(img_url, headers=req_header)
            if img_res.ok:
                file_ext = os.path.splitext(img_url)[-1]
                if not file_ext or len(file_ext) > 5:
                    file_ext = ".jpg"
                file_path = os.path.join(dir_name, f"{idx:03}{file_ext}")
                with open(file_path, 'wb') as file:
                    file.write(img_res.content)
                print(f"저장됨: {file_path}")
            else:
                print(f"다운 실패: {img_url}")
        except Exception as e:
            print(f"오류 ({img_url}): {e}")

# 호출 예시
download_one_episode('일렉시드', 341, 'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')


저장됨: img\일렉시드\341\001.jpg
저장됨: img\일렉시드\341\002.jpg
저장됨: img\일렉시드\341\003.jpg
저장됨: img\일렉시드\341\004.jpg
저장됨: img\일렉시드\341\005.jpg
저장됨: img\일렉시드\341\006.jpg
저장됨: img\일렉시드\341\007.jpg
저장됨: img\일렉시드\341\008.jpg
저장됨: img\일렉시드\341\009.jpg
저장됨: img\일렉시드\341\010.jpg
저장됨: img\일렉시드\341\011.jpg
저장됨: img\일렉시드\341\012.jpg
저장됨: img\일렉시드\341\013.jpg
저장됨: img\일렉시드\341\014.jpg
저장됨: img\일렉시드\341\015.jpg
저장됨: img\일렉시드\341\016.jpg
저장됨: img\일렉시드\341\017.jpg
저장됨: img\일렉시드\341\018.jpg
저장됨: img\일렉시드\341\019.jpg
저장됨: img\일렉시드\341\020.jpg
저장됨: img\일렉시드\341\021.jpg
저장됨: img\일렉시드\341\022.jpg
저장됨: img\일렉시드\341\023.jpg
저장됨: img\일렉시드\341\024.jpg
저장됨: img\일렉시드\341\025.jpg
저장됨: img\일렉시드\341\026.jpg
저장됨: img\일렉시드\341\027.jpg
저장됨: img\일렉시드\341\028.jpg
저장됨: img\일렉시드\341\029.jpg
저장됨: img\일렉시드\341\030.jpg
저장됨: img\일렉시드\341\031.jpg
저장됨: img\일렉시드\341\032.jpg
저장됨: img\일렉시드\341\033.jpg
저장됨: img\일렉시드\341\034.jpg
저장됨: img\일렉시드\341\035.jpg
저장됨: img\일렉시드\341\036.jpg
저장됨: img\일렉시드\341\037.jpg
저장됨: img\일렉시드\341\038.jpg
저장됨: img\일렉시